In [3]:
import bs4
from bs4 import BeautifulSoup
import requests
import re
import json
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold, train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor,\
AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier, \
RandomForestClassifier, BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor

In [4]:
# get list of top 250 movies from imdb
URL ="http://www.imdb.com/chart/top"
r = requests.get(URL)
movies_soup = BeautifulSoup(r.content, "lxml")

In [7]:
# create list of imdb ids and names
title_list = []
id_list = []
for movie in movies_soup.find_all('a', href=re.compile('/title/')):
    title_list.append(movie.text.strip())
    id_list.append(movie['href'])

In [23]:
pi = [i.split("/")[2] for i in id_list]
pi = 

500

In [30]:
pi_df = pd.DataFrame(pi)
pi_df.columns = ['id']
pi_df.drop_duplicates(inplace=True)
pi_df.head(3)

,id
0,tt0111161
2,tt0068646
4,tt0071562


In [31]:
id_2 = id_list

In [35]:
id_2 = 
for a, b in enumerate(id_list[0:10]):
    print a, b.split("/")[2]

0 tt0111161
1 tt0111161
2 tt0068646
3 tt0068646
4 tt0071562
5 tt0071562
6 tt0468569
7 tt0468569
8 tt0050083
9 tt0050083


In [38]:
list(enumerate(id_2))

[(0,
  '/title/tt0111161/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1RSZA9PQ596Q91X3QQH8&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_1'),
 (1,
  '/title/tt0111161/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1RSZA9PQ596Q91X3QQH8&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_1'),
 (2,
  '/title/tt0068646/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1RSZA9PQ596Q91X3QQH8&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_2'),
 (3,
  '/title/tt0068646/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1RSZA9PQ596Q91X3QQH8&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_2'),
 (4,
  '/title/tt0071562/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1RSZA9PQ596Q91X3QQH8&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_3'),
 (5,
  '/title/tt0071562/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1RSZA9PQ596Q91X3QQH8&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_3'),
 (6,
  '/title/tt0468569/?pf_rd_m=A2FGELUUNOQJ

In [ ]:
name_list

In [ ]:
id_list = [x.split("/")[2] for x in id_list]
id_list = [j for i,j in enumerate(id_list) if i%2]
name_list = [j.encode("utf-8") for i,j in enumerate(name_list) if i%2]

In [ ]:
len(name_list)

In [ ]:
# use id list to send requests to API
URL_base = "http://www.omdbapi.com/?i={}&plot=full&r=json&tomatoes=true"
# loop over id list to put json output in list
json_list = []
for i in id_list:
    r = requests.get(URL_base.format(i))
    json_list.append(json.loads(r.content))

In [ ]:
json_list[0]

In [ ]:
# create list of features to extract from json
cols = ["imdbRating", "Title","Rated","Country", "Genre", "Year","Awards","Runtime","Director",\
       "Actors","Plot","Metascore","imdbVotes", "Language","tomatoMeter","tomatoRotten","tomatoFresh"]

In [ ]:
# create dictionary of results for movies; key is id, value is dictionary of fields
movie_dict={}
for i, j in zip(id_list,json_list):
    temp_dict = {}
    for col in cols:
        temp_dict[col]=j[col]
    movie_dict[i] = temp_dict 

In [ ]:
# create df of data from API
movie_df = pd.DataFrame([k for k,v in movie_dict.iteritems()], columns=["id"])
for col in cols:
    movie_df = pd.merge(movie_df,pd.DataFrame([(k,v[col]) for k,v in movie_dict.iteritems()], columns=["id","var"]),\
                        on ="id", how="left")

In [ ]:
# rename columns
col_names = ["id"]
col_names.extend(cols)
movie_df.columns = col_names

In [ ]:
# visit imdb page for each movie (by imdb id) and collect content
URL_base = "http://www.imdb.com/title/{}"
imdb_list = []
for i in id_list:
    r = requests.get(URL_base.format(i))
    imdb_soup = BeautifulSoup(r.content, "lxml")
    imdb_list.append(imdb_soup)

In [ ]:
# functions to extract gross and budget from pages
def extract_gross(page):
    for i in page.find_all("h4", class_="inline"):
        if i.text == "Gross:":
            return i.next_sibling.strip()
def extract_budget(page):
    for i in page.find_all("h4", class_="inline"):
        if i.text == "Budget:":
            return i.next_sibling.strip()

In [ ]:
# scrape budget and gross from pages, create a df
gross_list = []
budget_list = []
for page in imdb_list:
    gross_list.append(extract_gross(page))
    budget_list.append(extract_budget(page))

imdb_df = pd.DataFrame(id_list, columns = ["id"])
imdb_df["Budget"] = budget_list
imdb_df["Gross"] = gross_list

In [ ]:
# merge the two dfs on id
df = pd.merge(movie_df, imdb_df, on="id", how="left")
df.head(3)